## Set-Up

This notebook connects to the MPMF_QC database (a local version), extracts all the tables as well as creating an RDS file of the data. 

In [54]:
# library for MySQL access
#install.packages("RMariaDB")
library(RMariaDB)
library(tidyverse)

Warning message:
"package 'tidyverse' was built under R version 3.6.3"-- Attaching packages --------------------------------------- tidyverse 1.3.0 --
v ggplot2 3.3.2     v purrr   0.3.4
v tibble  3.0.3     v dplyr   1.0.1
v tidyr   1.1.1     v stringr 1.4.0
v readr   1.3.1     v forcats 0.4.0
Warning message:
"package 'ggplot2' was built under R version 3.6.3"Warning message:
"package 'tibble' was built under R version 3.6.3"Warning message:
"package 'tidyr' was built under R version 3.6.3"Warning message:
"package 'purrr' was built under R version 3.6.3"Warning message:
"package 'dplyr' was built under R version 3.6.3"-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


In [2]:
# set password for db
localPassword <- "raja2417"

In [3]:
# connection
MPMF_QC_DB <- dbConnect(RMariaDB::MariaDB(), user='root', password=localPassword, dbname='mpmfdb', host='localhost')

# close connection
#dbDisconnect(MPMF_QC_DB)

In [4]:
# show tables
dbListTables(MPMF_QC_DB)

[1] "chromatogram"     "experiment"       "machine"          "measurement"     
[5] "metric"           "pressure_profile" "qc_run"           "sample_component"
[9] "stat"

## Machines

In [5]:
# get all machines query
query <- paste("SELECT *
    FROM machine;",sep="")

In [6]:
# get results of the query
rs <- dbSendQuery(MPMF_QC_DB, query)
machines <- dbFetch(rs)

In [7]:
machines

machine_id,machine_name,machine_serial,machine_description,machine_venue,use_metab,use_prot,machine_type
1,fusion,NA,NA,CLAYTON,N,Y,thermo
2,qeclassic,NA,NA,CLAYTON,Y,Y,thermo
3,qehf1,NA,NA,CLAYTON,N,Y,thermo
4,qeplus1,NA,NA,CLAYTON,N,Y,thermo
5,qeplus2,NA,NA,CLAYTON,N,Y,thermo
6,qecmips,NA,NA,PARKVILLE,Y,Y,thermo
7,qehfmips,NA,NA,PARKVILLE,N,Y,thermo


In [8]:
str(machines)

'data.frame':	7 obs. of  8 variables:
 $ machine_id         : int  1 2 3 4 5 6 7
 $ machine_name       : chr  "fusion" "qeclassic" "qehf1" "qeplus1" ...
 $ machine_serial     : chr  NA NA NA NA ...
 $ machine_description: chr  NA NA NA NA ...
 $ machine_venue      : chr  "CLAYTON" "CLAYTON" "CLAYTON" "CLAYTON" ...
 $ use_metab          : chr  "N" "Y" "N" "N" ...
 $ use_prot           : chr  "Y" "Y" "Y" "Y" ...
 $ machine_type       : chr  "thermo" "thermo" "thermo" "thermo" ...


## Metrics

In [9]:
# get all metrics query
query <- paste("SELECT *
    FROM metric;",sep="")

In [10]:
# get results of the query
rs <- dbSendQuery(MPMF_QC_DB, query)
metrics <-dbFetch(rs)

Warning message in result_create(conn@ptr, statement, is_statement):
"Cancelling previous query"

In [11]:
metrics

metric_id,metric_name,metric_description,display_order,display_name,use_metab,use_prot,metric_type
1,mz,mass to charge ratio,-1,Mass to Charge Ratio (mz),Y,Y,mzmine
2,rt,retention time,3,Retention Time,Y,Y,mzmine
3,height,spectra peak height,-1,Height,Y,Y,mzmine
4,area,spectra peak area,-1,Area,Y,Y,mzmine
5,fwhm,full width half maximum,6,Full Width Half Maximum,Y,Y,mzmine
6,tf,tailing factor,7,Tailing,Y,Y,mzmine
7,af,asymmetry factor,8,Asymmetry,Y,Y,mzmine
8,mz_min,mass to charge minimum,-1,mz Minimum,Y,Y,mzmine
9,mz_max,mass to charge maximum,-1,mz Maximum,Y,Y,mzmine
10,mass_error_ppm,relative error in parts per million,1,Mass Error (ppm),Y,Y,mzmine


In [12]:
str(metrics)

'data.frame':	35 obs. of  8 variables:
 $ metric_id         : int  1 2 3 4 5 6 7 8 9 10 ...
 $ metric_name       : chr  "mz" "rt" "height" "area" ...
 $ metric_description: chr  "mass to charge ratio" "retention time" "spectra peak height" "spectra peak area" ...
 $ display_order     : int  -1 3 -1 -1 6 7 8 -1 -1 1 ...
 $ display_name      : chr  "Mass to Charge Ratio (mz)" "Retention Time" "Height" "Area" ...
 $ use_metab         : chr  "Y" "Y" "Y" "Y" ...
 $ use_prot          : chr  "Y" "Y" "Y" "Y" ...
 $ metric_type       : chr  "mzmine" "mzmine" "mzmine" "mzmine" ...


## Components

In [13]:
# get all components query
query <- paste("SELECT *
    FROM sample_component;",sep="")

In [14]:
# get results of the query
rs <- dbSendQuery(MPMF_QC_DB, query)
components <-dbFetch(rs)

Warning message in result_create(conn@ptr, statement, is_statement):
"Cancelling previous query"

In [15]:
components

component_id,component_name,component_description,component_mode,exp_mass_charge,exp_rt,experiment_id
1,Lactic_acid,NA,N,89.02442,9.96,2
2,Succinic_acid,NA,N,117.01933,15.50,2
3,Leucine,NA,N,130.08735,10.20,2
4,Isoleucine,NA,N,130.08735,11.00,2
5,Orotic_acid,NA,N,155.00983,10.80,2
6,HEPES,NA,N,237.09145,10.69,2
7,G6P,NA,N,259.02244,16.79,2
8,Taurocholic_acid,NA,N,514.28440,4.46,2
9,CHAPS,NA,N,613.38920,7.88,2
10,EPPS,NA,N,251.10710,10.72,2


In [16]:
str(components)

'data.frame':	31 obs. of  7 variables:
 $ component_id         : int  1 2 3 4 5 6 7 8 9 10 ...
 $ component_name       : chr  "Lactic_acid" "Succinic_acid" "Leucine" "Isoleucine" ...
 $ component_description: chr  NA NA NA NA ...
 $ component_mode       : chr  "N" "N" "N" "N" ...
 $ exp_mass_charge      : num  89 117 130 130 155 ...
 $ exp_rt               : num  9.96 15.5 10.2 11 10.8 ...
 $ experiment_id        : int  2 2 2 2 2 2 2 2 2 2 ...


## Experiments

In [17]:
# get all experiments query
query <- paste("SELECT *
    FROM experiment;",sep="")

In [18]:
# get results of the query
rs <- dbSendQuery(MPMF_QC_DB, query)
experiments <-dbFetch(rs)

Warning message in result_create(conn@ptr, statement, is_statement):
"Cancelling previous query"

In [19]:
experiments

experiment_id,experiment_type
1,proteomics
2,metabolomics


## Chromatograms

In [47]:
# get all chroms query (cast JSON to CHAR)
query <- paste("SELECT cast(chrom_data as CHAR) as 'Chrom. JSON', run_id, component_id
    FROM chromatogram;",sep="")

# code needed to extract and convert back to JSON

In [48]:
# get results of the query
rs <- dbSendQuery(MPMF_QC_DB, query)
chromatograms <-dbFetch(rs)

Warning message in result_create(conn@ptr, statement, is_statement):
"Cancelling previous query"

In [49]:
str(chromatograms)

'data.frame':	1587 obs. of  3 variables:
 $ Chrom. JSON : chr  "{\"mz\": 90.05516578291223, \"rts\": [1, 3, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 11, 1, 2, 1, 7, 2, 1, 2, 2, 1, 1, "| __truncated__ "{\"mz\": 119.06066851161413, \"rts\": [1, 1, 1, 1, 11, 1, 1, 6, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 2, 3, 2, 1, 4,"| __truncated__ "{\"mz\": 122.08145858522016, \"rts\": [1, 1, 36, 2, 1, 11, 1, 2, 5, 1, 1, 33, 1, 1, 9, 1, 2, 8, 2, 1, 31, 1, 2,"| __truncated__ "{\"mz\": 166.0867021040483, \"rts\": [1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1"| __truncated__ ...
 $ run_id      : int  7690 7690 7690 7690 7690 7690 7690 7690 7690 7690 ...
 $ component_id: int  11 12 13 14 16 17 18 1 2 3 ...


## Pressure Profiles

In [50]:
# get all pressure profiles query (cast JSON to CHAR)
query <- paste("SELECT cast(pressure_data as CHAR) as 'Pressure JSON', run_id, pump_type
    FROM pressure_profile;",sep="")

# code needed to extract and convert back to JSON

In [51]:
# get results of the query
rs <- dbSendQuery(MPMF_QC_DB, query)
profiles <-dbFetch(rs)

Warning message in result_create(conn@ptr, statement, is_statement):
"Cancelling previous query"

In [52]:
str(profiles)

'data.frame':	219 obs. of  3 variables:
 $ Pressure JSON: chr  "{\"rts\": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1"| __truncated__ "{\"rts\": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1"| __truncated__ "{\"rts\": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1"| __truncated__ "{\"rts\": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1"| __truncated__ ...
 $ run_id       : int  7690 7691 7692 7693 7694 7695 7696 7735 7736 7737 ...
 $ pump_type    : chr  "mp" "mp" "mp" "mp" ...


## Metric Measurements

In [32]:
# get all metric measurements query (ie the data)
query <- paste("SELECT m.display_name as 'metric', m.metric_type, s.component_name, e.experiment_type, h.machine_name, q.date_time, v.value
    FROM measurement v, sample_component s, qc_run q, metric m, experiment e, machine h 
    WHERE v.run_id = q.run_id AND 
            v.component_id = s.component_id AND 
            v.metric_id = m.metric_id AND 
            e.experiment_id = q.experiment_id AND 
            e.experiment_id = s.experiment_id AND 
            q.machine_id = h.machine_id;",sep="")

In [33]:
# get results of the query
rs = dbSendQuery(MPMF_QC_DB, query)
mpmf_data <-dbFetch(rs)

Warning message in result_create(conn@ptr, statement, is_statement):
"Cancelling previous query"

In [34]:
head(mpmf_data)

metric,metric_type,component_name,experiment_type,machine_name,date_time,value
Mass to Charge Ratio (mz),mzmine,iRT-pep_a,proteomics,fusion,2019-08-30 01:14:55,487.2569
Mass to Charge Ratio (mz),mzmine,iRT-pep_b,proteomics,fusion,2019-08-30 01:14:55,644.8241
Mass to Charge Ratio (mz),mzmine,iRT-pep_c,proteomics,fusion,2019-08-30 01:14:55,683.8297
Mass to Charge Ratio (mz),mzmine,iRT-pep_d,proteomics,fusion,2019-08-30 01:14:55,547.2996
Mass to Charge Ratio (mz),mzmine,iRT-pep_e,proteomics,fusion,2019-08-30 01:14:55,669.8397
Mass to Charge Ratio (mz),mzmine,iRT-pep_f,proteomics,fusion,2019-08-30 01:14:55,683.8551


In [53]:
str(mpmf_data)

'data.frame':	1161313 obs. of  7 variables:
 $ metric         : Factor w/ 32 levels "Air Injection",..: 14 14 14 14 14 14 14 14 14 14 ...
 $ metric_type    : Factor w/ 3 levels "morpheus","mzmine",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ component_name : Factor w/ 31 levels "Alanine","Benzimidazole",..: 10 11 12 13 14 15 16 17 18 19 ...
 $ experiment_type: Factor w/ 2 levels "metabolomics",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ machine_name   : Factor w/ 7 levels "fusion","qeclassic",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ date_time      : POSIXct, format: "2019-08-30 01:14:55" "2019-08-30 01:14:55" ...
 $ value          : num  487 645 684 547 670 ...


### Wrangle and Save

In [37]:
# convert to factors
mpmf_data$metric <- as.factor(mpmf_data$metric)
mpmf_data$component_name <- as.factor(mpmf_data$component_name)
mpmf_data$experiment_type <- as.factor(mpmf_data$experiment_type)
mpmf_data$machine_name <- as.factor(mpmf_data$machine_name)
mpmf_data$metric_type <- as.factor(mpmf_data$metric_type)

In [57]:
levels(mpmf_data$metric)

[1] "Air Injection"                       "Area"                               
 [3] "Area (normalised)"                   "Asymmetry"                          
 [5] "Column Oven"                         "End Pressure (Average)"             
 [7] "End Pressure (Median)"               "Full Width Half Maximum"            
 [9] "Height"                              "Height (normalised)"                
[11] "Inline Leak"                         "Mass Error (mDa)"                   
[13] "Mass Error (ppm)"                    "Mass to Charge Ratio (mz)"          
[15] "Maximum Pressure"                    "Minimum Pressure"                   
[17] "MS/MS Spectra (Morpheus)"            "mz Maximum"                         
[19] "mz Minimum"                          "Precursor Mass Error ppm (Morpheus)"
[21] "Pressure Differential"               "Protein Groups (Morpheus)"          
[23] "PSM (Morpheus)"                      "Retention Time"                     
[25] "Retention Time at Maximum Pressure"  "Starting Backpressure (Average)"    
[27] "Starting Pressure (Median)"          "Tailing"                            
[29] "Unique Peptides (Morpheus)"          "Valve Drop"                         
[31] "Valve Spike End"                     "Valve Spike Start"

In [55]:
unused_metrics <- c("Area", "Height", "mz Maximum", "mz Minimum", "Mass to Charge Ratio (mz)")

In [56]:
mpmf_data <- filter(mpmf_data, !mpmf_data$metric %in% unused_metrics)

In [58]:
str(mpmf_data)

'data.frame':	727743 obs. of  7 variables:
 $ metric         : Factor w/ 32 levels "Air Injection",..: 24 24 24 24 24 24 24 24 24 24 ...
 $ metric_type    : Factor w/ 3 levels "morpheus","mzmine",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ component_name : Factor w/ 31 levels "Alanine","Benzimidazole",..: 10 11 12 13 14 15 16 17 18 19 ...
 $ experiment_type: Factor w/ 2 levels "metabolomics",..: 2 2 2 2 2 2 2 2 2 2 ...
 $ machine_name   : Factor w/ 7 levels "fusion","qeclassic",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ date_time      : POSIXct, format: "2019-08-30 01:14:55" "2019-08-30 01:14:55" ...
 $ value          : num  19.6 21.8 24.2 25.6 26.7 ...


In [60]:
head(mpmf_data)

metric,metric_type,component_name,experiment_type,machine_name,date_time,value
Retention Time,mzmine,iRT-pep_a,proteomics,fusion,2019-08-30 01:14:55,19.57583
Retention Time,mzmine,iRT-pep_b,proteomics,fusion,2019-08-30 01:14:55,21.82167
Retention Time,mzmine,iRT-pep_c,proteomics,fusion,2019-08-30 01:14:55,24.17033
Retention Time,mzmine,iRT-pep_d,proteomics,fusion,2019-08-30 01:14:55,25.59117
Retention Time,mzmine,iRT-pep_e,proteomics,fusion,2019-08-30 01:14:55,26.73767
Retention Time,mzmine,iRT-pep_f,proteomics,fusion,2019-08-30 01:14:55,27.67833


In [61]:
# save as R object
saveRDS(mpmf_data, file="MPMF_QC_DB.rds")